**PMR3508 - Tarefa 2 -
Classificador de Spam ou Ham**

In [ ]:
# Importando principais bibliotecas a serem utilizadas.

import sklearn # algoritmos para tratar os dados
import matplotlib.pyplot as plt #gráficos
import numpy as np # álgebra linear
import pandas as pd # processamento de dados
import scikitplot as skplt
from sklearn.model_selection import cross_val_score

Primeiro, nomeando a base de treino e entendendo os dados da spambase.

In [ ]:
treino = pd.read_csv("../input/spambase/train_data.csv" , engine='python')
teste = pd.read_csv("../input/spambase/test_features.csv" , engine='python')
treino.head()

In [ ]:
treino.describe()

Percebo na tabela acima, de estatística destritiva, que algumas palavras tem mais variação quanto a média, outras menos, e algumas outras informações podem ser obtidas.  Verificar as frequências de algumas palavras chave, dentre e-mails spam ou ham, pode ser muito eficiente para diferenciá-los. Então, como partida, observar a correlação entre frequência de algumas palavras e o fato do e-mail ser ham. 

In [ ]:
treino.corr(method='pearson').ham.sort_values(ascending=True)

In [ ]:
tab = pd.crosstab(treino['ham'], treino['char_freq_$'], margins = True,
normalize = 'columns')
tab

Na tabela acima, informações bem relevantes estão nas primeira e última colunas, que são 0.0 e All. Podemos perceber que a ausência do caractere $ diz muito sobre o e-mail ser spam ou ham. Esse caractere, em algumas frequências, como 0.030 e 0.032 tem mais de 70 porcento de ocorrência de spam, enquanto 0.035 tem ocorrência total de spam, e 0.033 tem total ocorrência de ham. É preciso tomar alguns cuidados com as faixas de porcentagem para o reconhecimento de padrões, pois algumas tem poucos exemplos de e-mails com essa característica. Sem normalizar:

In [ ]:
tab2 = pd.crosstab(treino['ham'], treino['char_freq_$'], margins = True)
tab2

In [ ]:
tab3 = pd.crosstab(treino['ham'], treino['word_freq_hp'], margins = True, normalize = "columns")
tab3

Entendendo o problema : verificando quantos ham e quantos spam. O objetivo é separá-los e classificar de acordo com as características que mais os diferenciam. Para começar, dividir os conjuntos de dados entre spam e não spam.

In [ ]:
treino["ham"].value_counts()

In [ ]:
_ham = treino.query("ham").drop(columns=["Id"])
_ham["ham"].value_counts()


In [ ]:
_spam = treino.query("ham == False").drop(columns=["Id"])
_spam["ham"].value_counts()

In [ ]:
_spam.describe()

In [ ]:
_ham.describe()

In [ ]:
data1 = _spam["word_freq_your"]
data2 = _spam["word_freq_000"]
data3 = _spam["char_freq_$"]
x = np.array(range(len(data1)))

plt.plot( x, data1,  color='grey', label='your')
plt.plot( x, data2,  color='blue', label = '000')
plt.plot( x, data3,  color='yellow', label = '$') 

plt.rcParams['figure.figsize'] = (10,7)


plt.title("Relação de frequências com ser spam")
plt.legend()

plt.grid(True)
plt.xlabel("e-mails spam")
plt.ylabel("frequência de your, 000, $ ")
plt.show()

In [ ]:
data1 = _ham["word_freq_your"]
data2 = _ham["word_freq_000"]
data3 = _ham["char_freq_$"]
x = np.array(range(len(data1)))

plt.plot( x,data1,  color='grey', label='your') 
plt.plot( x, data2,  color='blue', label = '000')  
plt.plot( x, data3,  color='yellow', label = '$')  

plt.rcParams['figure.figsize'] = (10,7)

plt.legend()
plt.title("Relação de frequências com ser ham")

plt.grid(True)
plt.xlabel("e-mails ham")
plt.ylabel("frequência de your, 000, $ ")
plt.show()

In [ ]:
data1 = _ham["word_freq_000"]
data2 = _spam["word_freq_000"]
data3 = _ham["char_freq_$"]
data4 = _spam["char_freq_$"]
x = np.array(range(len(data1)))
w = np.array(range(len(data2)))

plt.plot( w, data2  , color='purple', label = '000 spam') 
plt.plot( w, data4 ,color='pink', label = '$ spam') 
plt.plot( x, data3 ,color='grey', label = '$ ham')  
plt.plot( x, data1, color='cyan', label='000 ham') 

plt.rcParams['figure.figsize'] = (10,7)

plt.legend()
plt.title("Relação de frequências com ser ham ou spam")

plt.grid(True)
plt.xlabel("e-mails")
plt.ylabel("frequência de 000, $ ")
plt.show()

Nas tabelas acima, foi possível demonstrar que certos termos, como 000 e $ se apresentam de forma bem distinta nos e-mails spam ou ham.

A partir disso, vou agora separar os labels, rótulos, dos features, atributos, para começar o método de Naive Bayes utilizando distribuição gaussiana, bernoulli e multinomial, com média e desvio estimados utilizando máxima verossimilhança. (implementado pela biblioteca sklearn.naive_bayes)

Uma métrica muito interessante para se ter é a matriz de confusão para cada classificador.

In [ ]:
#gaussiana sem Id
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
rtr = treino["ham"]                                  #rotulos treino
atr_si = treino.drop(columns=["ham","Id"])           #atributos treino
y_pred_gnb = gnb.fit(atr_si, rtr).predict(atr_si)
print("Número de pontos falsamente rotulados (mislabeled points) de um total de %d pontos : %d" % (atr_si.shape[0],(rtr != y_pred_gnb).sum()))
print("Erro empírico: {:.2%}" .format((rtr != y_pred_gnb).sum()/atr_si.shape[0]))
scores = cross_val_score(gnb, atr_si, rtr, cv=10, n_jobs=-1, scoring='roc_auc')
print("Score por cross validation em 10 folds : {:.2%} ".format(np.mean(scores)))
skplt.metrics.plot_confusion_matrix(rtr, y_pred_gnb,cmap="cool",figsize=(6,6))

In [ ]:
#gaussiana com Id
rtr = treino["ham"]                               #rotulos treino
atr = treino.drop(columns=["ham"])                #atributos treino
y_pred_gnb2 = gnb.fit(atr, rtr).predict(atr)
print("Número de pontos falsamente rotulados (mislabeled points) de um total de %d pontos : %d" % (atr.shape[0],(rtr != y_pred_gnb2).sum()))
print("Erro empírico: {:.2%}" .format((rtr != y_pred_gnb2).sum()/atr.shape[0]))
scores = cross_val_score(gnb, atr, rtr, cv=10, n_jobs=-1, scoring='roc_auc')
print("Score por cross validation em 10 folds : {:.2%} ".format(np.mean(scores)))
skplt.metrics.plot_confusion_matrix(treino["ham"], y_pred_gnb2,cmap="cool",figsize=(6,6))

Acima, pude perceber que a presenca da coluna Id atrapalhou o desempenho do classificador gaussiano. Ainda assim, o erro esta muito alto, entao tentarei outra estrategia: em vez de gaussiana, utilizar o naive bayes para distribuicoes de bernoulli. Abaixo, vemos que de fato o erro empirico diminuiu bastante, porem o Id nao faz diferenca, pois sua "binarizacao" seria a mesma para todos os e-mails e igual a 1.

In [ ]:
from sklearn.naive_bayes import BernoulliNB
bnb = BernoulliNB()
y_pred = bnb.fit(atr, rtr).predict(atr)
print("Número de pontos falsamente rotulados (mislabeled points) de um total de %d pontos : %d" % (atr.shape[0],(rtr != y_pred).sum()))
print("Erro empírico: {:.2%}" .format((rtr != y_pred).sum()/atr.shape[0]))
scores = cross_val_score(bnb, atr, rtr, cv=10, n_jobs=-1, scoring='roc_auc')
print("Score por cross validation em 10 folds : {:.2%} ".format(np.mean(scores)))
skplt.metrics.plot_confusion_matrix(treino["ham"], y_pred,cmap="cool",figsize=(6,6))

In [ ]:
from sklearn.naive_bayes import MultinomialNB
mnb = MultinomialNB()
y_pred_mnb = mnb.fit(atr_si, rtr).predict(atr_si)
print("Número de pontos falsamente rotulados (mislabeled points) de um total de %d pontos : %d" % (atr_si.shape[0],(rtr != y_pred_mnb).sum()))
print("Erro empírico: {:.2%}" .format((rtr != y_pred_mnb).sum()/atr_si.shape[0]))
scores = cross_val_score(mnb, atr_si, rtr, cv=10, n_jobs=-1, scoring='roc_auc')
print("Score por cross validation em 10 folds : {:.2%} ".format(np.mean(scores)))
skplt.metrics.plot_confusion_matrix(treino["ham"], y_pred_mnb,cmap="cool",figsize=(6,6))

Concluo que o melhor classificador, dentre os Naive Bayes testados, é o Bernoulli Naive Bayes. Além de score mais alto e mais acertos empíricos, também obteve o melhor desempenho na matriz de confusão, ou seja, cometeu bem menos do "pior tipo de erro", em que o classificador moveria um e-mail importante para a caixa de spam.

Finalmente, rumo a outra forma de classificação, a **kNN**. Testei diversos parâmetros para a função K vizinhos mais próximos, até obter um inacreditável erro próximo de zero. Dentre possíveis métricas, algumas testadas foram canberra, sokalmichener, jaccard e dice. Primeiro, a função com seus parâmetros alterados, e depois, com parâmetros Default para comparação:

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(algorithm='brute', leaf_size=30, metric="canberra",
           metric_params=None, n_jobs=8, n_neighbors=16,
           weights='distance')


y_pred_knn = knn.fit(atr_si,rtr).predict(atr_si)
print("Número de pontos falsamente rotulados (mislabeled points) de um total de %d pontos : %d" % (atr_si.shape[0],(rtr != y_pred_knn).sum()))
print("Erro empírico: {:.2%}" .format((rtr != y_pred_knn).sum()/atr_si.shape[0]))
scores = cross_val_score(knn, atr_si, rtr, cv=10, n_jobs=8, scoring='roc_auc')
print("Score por cross validation em 10 folds : {:.3%} ".format(np.mean(scores)))
skplt.metrics.plot_confusion_matrix(rtr, y_pred_knn,cmap="YlOrRd",figsize=(6,6))

In [ ]:
knn2 = KNeighborsClassifier(n_neighbors=5, weights="uniform", 
                           algorithm="auto", leaf_size=30, p=2, 
                           metric="minkowski", metric_params=None, n_jobs=None)

y_pred_knn2 = knn2.fit(atr, rtr).predict(atr)
print("Número de pontos falsamente rotulados (mislabeled points) de um total de %d pontos : %d" % (atr.shape[0],(rtr != y_pred_knn2).sum()))
print("Erro empírico: {:.2%}" .format((rtr != y_pred_knn2).sum()/atr.shape[0]))
scores = cross_val_score(knn2, atr, rtr, cv=5, n_jobs=-1, scoring='roc_auc')
print("Score por cross validation em 10 folds : {:.2%} ".format(np.mean(scores)))
skplt.metrics.plot_confusion_matrix(rtr, y_pred_knn2,cmap="YlOrRd",figsize=(6,6))

In [ ]:
y_teste1 = knn.fit(atr_si,rtr).predict(teste.drop(columns="Id"))
y_probas1 = bnb.fit(atr_si,rtr).predict_proba(teste.drop(columns="Id"))
skplt.metrics.plot_roc(y_teste1,y_probas1,classes_to_plot =None,cmap="spring",)
plt.show()

In [ ]:
y_teste2= bnb.fit(atr,rtr).predict(teste)
y_probas2 = knn.fit(atr,rtr).predict_proba(teste)
skplt.metrics.plot_roc(y_teste2,y_probas2,classes_to_plot =None,cmap="spring",)
plt.show()

Este Notebook utilizou as seguintes referências:
*     Python Sotfware Foundation
*     Naive Bayes Scikit Learn
*     Pandas Documentation
* MatPlotLib.org
* Scikit Learn Documentation 


In [ ]:
submit = pd.DataFrame({'Id':teste["Id"],'ham':y_teste1[:]})
submit.to_csv("y_teste4.csv", index = False)
submit.head()

In [ ]:
def facilitador(metrica,vizinhos,pe,nome):
    knn3 = KNeighborsClassifier(algorithm='auto', leaf_size=30, metric=metrica,
               metric_params=None, n_jobs=8, n_neighbors=vizinhos, p = pe,
               weights='distance')

    y_pred_knn3 = knn3.fit(atr_si,rtr).predict(atr_si)

    print("Número de pontos falsamente rotulados (mislabeled points) de um total de %d pontos : %d" % (atr_si.shape[0],(rtr != y_pred_knn3).sum()))
    print("Erro empírico: {:.2%}" .format((rtr != y_pred_knn3).sum()/atr_si.shape[0]))
    
    scores = cross_val_score(knn3, atr_si, rtr, cv=10, n_jobs=8, scoring='roc_auc')
    print("Score por cross validation em 10 folds : {:.3%} ".format(np.mean(scores)))
    

    predtest = knn3.fit(atr_si,rtr).predict(teste.drop(columns="Id"))

    submit = pd.DataFrame({'Id':teste["Id"],'ham':predtest[:]})
    submit.to_csv(nome, index = False)
    
facilitador("rogerstanimoto",15,2, "predicoes.csv")

In [ ]:
facilitador("sokalsneath",16,1,"predsokal.csv")

In [ ]:
facilitador("rogerstanimoto",18,1,"predroger.csv")

In [ ]:
facilitador("minkowski",10,1,"predmink.csv")